https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e

In [1]:
!pip install pandas matplotlib noos-pyk

In [2]:
import datetime as dt

import pandas as pd
import plotly.graph_objects as go

from dash_mdc_neptune import themes

In [3]:
from http import client as http_client
from noos_pyk.clients import json


class NeptuneClient(json.JSONClient):

    default_base_url = "https://api.noos.energy/"

    def retrieve_time_series(self, curve_type, curve_uid, published_at = None):
        params = None
        if published_at:
            params = {"published_at": published_at.isoformat()}
        response = self.get(
            path=f"v1/curves/{curve_type}/contracts/{curve_uid}",
            params=params,
            statuses=(http_client.OK,),
        )
        return pd.DataFrame(data=response["time_series"])

In [4]:
# Global variables

TODAY = pd.to_datetime(dt.date.today(), utc=True)
CURVE_UID = "FWD_PWRTE_H_DA"
PUBLISHED_AT = TODAY - pd.tseries.offsets.Day(1)

In [5]:
# Retrieve auctions

query_params = {
    "curve_uid": CURVE_UID,
    "curve_type": "FIXING",
    "published_at": PUBLISHED_AT,
}

client = NeptuneClient()
df = client.retrieve_time_series(**query_params)

In [6]:
# Format auctions

df_da = df.copy()
df_da["timestamp"] = pd.to_datetime(df_da.timestamp, utc=True)
df_da = df_da.rename(columns={"timestamp": "delivery_from", "value": "price"})

# Add auction date and period
df_da["settlement_date"] = df_da.delivery_from.dt.date
df_da["settlement_period_id"] =  df_da.delivery_from.dt.hour

# Add local delivery
df_da["delivery_from_local"] = df_da.delivery_from.dt.tz_convert("Europe/Paris")

In [7]:
# Extract latest auctions

df_latest_da = df_da.tail(24)
df_latest_da = df_latest_da[
    ["settlement_date", "settlement_period_id", "price", "delivery_from_local"]
]
df_latest_da = df_latest_da.rename(
    columns={
        "settlement_date": "auction date",
        "settlement_period_id": "auction period",
        "price": "€/MWh",
        "delivery_from_local": "local delivery",
    }
)
# df_latest_da = df_latest_da.set_index("auction period")

# Format table

df_styler = df_latest_da.style
df_styler = df_styler.hide_index()
df_styler = df_styler.background_gradient(
    axis=0, gmap=df_latest_da["€/MWh"], cmap=themes.matplotlib_templates.noos_diverging
)
df_styler = df_styler.format({"€/MWh": "{:,.2f}", "hour": "{:,.0f}"})
df_styler = df_styler.set_table_attributes("style='font-size: 0.8rem'")

In [8]:
# Shared heatmap configs

GO_LAYOUT = go.Layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="local delivery date",
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="local delivery hour",
        )
    ),
    template="noos_watermark+noos_base+noos_colorscale+min_modebar+hoover_xunified",
)

GO_HEATMAP_KWARGS = {
    "colorscale": themes.NOOS_PALETTE_DIVERGING,
    "colorbar": {"title": "€/MWh"},
    "hovertemplate": (
        "delivery date: %{x:%Y-%m-%d}<br>"
        "delivery hour: %{y:.0f}<br>"
        "price: %{z:.2f}<extra></extra>"
    ),
}

In [9]:
# Extract heatmap data

df_heatmap_da = df_da.set_index("delivery_from_local")

df_heatmap_da["date"] = df_heatmap_da.index.date.astype("str")
df_heatmap_da["hour"] = df_heatmap_da.index.hour


# Quarterly heatmap

qtd_date = PUBLISHED_AT - pd.tseries.offsets.Week(12)
df_qtd_da = df_heatmap_da[qtd_date.date().isoformat() :]

qtd_data = go.Heatmap(z=df_qtd_da.price, x=df_qtd_da.date, y=df_qtd_da.hour, **GO_HEATMAP_KWARGS)

qtd_fig = go.FigureWidget(data=qtd_data, layout=GO_LAYOUT)

In [23]:
from dash import dcc
from dash import html
import dash_dangerously_set_inner_html as raw_html
import dash_mdc_neptune as mdc


canvas_style = {
    "display": 'grid',
    "gridTemplateColumns": "2fr 4fr",
    "gap": 2,
    "gridTemplateRows": "auto",
    "gridTemplateAreas": "table graph",
}

table = html.Div(children=raw_html.DangerouslySetInnerHTML(df_styler.to_html()))
table_paper = mdc.Paper(id="table_paper", children=[table])
table_box = mdc.Box(id="table_box", sx={"gridArea": "table"}, children=[table_paper])

graph = dcc.Graph(id="quarterly-heatmap", figure=qtd_fig, responsive=True)
graph_paper = mdc.Paper(id="graph_paper", children=[graph])
graph_box = mdc.Box(id="graph_box", sx={"gridArea": "graph"}, children=[graph_paper])

canvas_box = mdc.Box(id="canvas_box", sx=canvas_style, children=[table_box, graph_box])

layout = html.Div(style={'backgroundColor': themes.NOOS_HTML_COLORS["primary"]}, children=[canvas_box])

In [24]:
from jupyter_dash import JupyterDash


app = JupyterDash(__name__)
app.layout = layout

app.run_server(mode='jupyterlab', port=8051)

/Users/epieuchot/Checkouts/noos/dash-mdc-neptune/.venv/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

